## Project setup, imports, etc

In [11]:
# approach 2 - PRAW
import requests
import praw
import pandas as pd
import hashlib
import json
from datetime import datetime
from dateutil import tz
from collections import defaultdict

# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
CLIENT_ID = 'Mx4J1DKZ856aT1ug9glOdA'
SECRET_TOKEN = 'j0a57wDUiC9TSDQhx1CYNz4jl65Aiw'
USERNAME = 'malconst'
PASSWORD = '7(>vAn!GqC1"'
USER_AGENT = 'Seeker/0.0.1'

In [2]:
# Establish authorized Reddit instance
reddit = praw.Reddit(
    client_id = CLIENT_ID,
    client_secret = SECRET_TOKEN,
    user_agent = USER_AGENT,
    username = USERNAME,
    password = PASSWORD
)

## Search Reddit
PRAW Search doucmentation https://praw.readthedocs.io/en/latest/code_overview/models/subreddit.html?highlight=search#praw.models.Subreddit.search

*Important:* the search function in this implementation is a method on the Subreddit object, not Subreddits

For now, implemented hashing as described here https://www.codegrepper.com/code-examples/python/python+convert+string+to+hash

https://pythonspot.com/save-a-dictionary-to-a-file/

https://www.geeksforgeeks.org/how-to-read-dictionary-from-file-in-python/

https://stackoverflow.com/questions/2132985/how-to-import-or-include-data-structures-e-g-a-dict-into-a-python-file-from-a

In [57]:
# Variable setup and stuff
subreddit = reddit.subreddit("mechmarket") #set the subreddit object

# Access the database (local json object)
with open('dbdict.json') as f: #load it to f
    t = f.read() #read it to t
stored_search_test = json.loads(t) #turn t into a dictionary and store it

# Variable setup
current_hash_list = [] #to store current db hashes
searches = [] #holder for final storage; kept searches only
all_searches = [] #holder for analysis; all searches
search_queries = ["olivia","9009","hammerhead"] #list of the queries to search for (move to config.json)
limit = 3 #limit for the queries (move to config.json)
i_len = len(stored_search_test['searches']) #length of the current db searches

#store db hashes
for i in range(i_len):
    j_len = len(stored_search_test['searches'][i]) #length of the j loop for some reason this works lol
    for j in range(j_len):
        k_len = len(stored_search_test['searches'][i][j]['search_results'])
        for k in range(k_len):
            current_hash_list.append(stored_search_test['searches'][i][j]['search_results'][k]['body']['hash']) #store the hash to the current_hash_list


In [ ]:
# Search
for query in search_queries: #work through the list of queries
    results = [] #holder for handling the sets of search results stemming from each query
    for result in subreddit.search(query,sort="new", limit=limit): #make a hash in the same way as 
        # Structure the data before storing - there ahs to be a better way than this... like convering to JSON upstream
        result_id = str(result)
        to_hash = str(result.author)+ str(result.title)+ str(result.url)
        hash_object = hashlib.sha256(to_hash.encode('utf-8')) #
        hex_dig = hash_object.hexdigest() #the hash for this search result
        # all_searches.append({'result_id': result_id, 'body': body}) #could delete this, don't really need it

        #Evaluate the new hash against the hash in the database search results
        
        # Check to see if this search result's hash is in the list. If yes ignore it, if it's not, add it to the db
        if hex_dig in current_hash_list:
            print('already')
            break
        elif hex_dig not in current_hash_list:
            print('else if')
            body = {
                'author':result.author.name,
                'title':result.title,
                'created date': datetime.fromtimestamp(result.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                'url':result.url,
                'hash':hex_dig
            }
            results.append({'result_id': result_id, 'body': body}) #just bundle the result_id and body into its own result object and add it to the the results list
    if results:
        searches.append({'query':query,'search_results':results})
if searches:
    stored_search_test['searches'].append(searches) # update the stored_dictionary

In [59]:
# Store dictionary somewhere
# Save dictionary
data = json.dumps(stored_search_test) #the final updated database
f = open("dbdict.json","w")
f.write(data)
f.close()

## Graveyard

In [ ]:
# #Approach 1 - 
# import requests
# import praw

# # note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
# CLIENT_ID = 'Mx4J1DKZ856aT1ug9glOdA'
# SECRET_TOKEN = 'j0a57wDUiC9TSDQhx1CYNz4jl65Aiw'
# USERNAME = 'malconst'
# PASSWORD = '7(>vAn!GqC1"'
# USER_AGENT = 'Seeker/0.0.1'

# auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_TOKEN)

# # here we pass our login method (password), username, and password
# data = {'grant_type': 'password',
#         'username': USERNAME,
#         'password': PASSWORD}

# # setup our header info, which gives reddit a brief description of our app
# headers = {'User-Agent': 'Seeker/0.0.1'}

# # send our request for an OAuth token
# res = requests.post('https://www.reddit.com/api/v1/access_token',
#                     auth=auth, data=data, headers=headers)

# # convert response to JSON and pull access_token value
# TOKEN = res.json()

# # add authorization to our headers dictionary
# headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# res = requests.get("https://oauth.reddit.com/r/python/hot",
#                    headers=headers).json()

# print(res.url)  # let's see what we get

